In [1]:
import sys 
sys.path.append('../../../')
sys.path.append('../../')
sys.path.append('../')

from omegaconf import OmegaConf
from src.core.passive_learning import *
from src.core.auto_labeling import *
from src.utils.logging_utils import * 
from src.data_layer.datasets.dataset_factory import * 
from src.data_layer.dataset_utils import * 
from src.utils.counting_utils import *  #
from src.utils.common_utils import * 
from src.utils.vis_utils import *
#from src.core.run_lib import * 
from src.core.self_training import * 
import copy 
import random 

config_file = '../../configs/self-training/cifar10_med_net_base_conf.yaml'
conf = OmegaConf.load(config_file)

logger = get_logger('../../temp/logs/self-training-cifar10_mednet.log','PL',level=logging.DEBUG)
#logger = get_logger('../../temp/logs/pl.log','PL')

conf['eval'] = 'full'
conf['calib_conf'] = None 

In [2]:
q_conf = conf["train_pts_query_conf"] 

q_conf["seed_train_size"] = 4000
q_conf["max_num_train_pts"]= 10000

q_conf["query_batch_size"] = 2000
q_conf["query_strategy_name"] =  "margin_random_v2" 
q_conf["margin_random_v2_constant"]= 2


q_conf['include_pseudo_labeled'] = True 

q_conf['enable_active_querying'] = True 

conf['stopping_criterion'] = "max_epochs"

In [3]:
auto_lbl_conf = conf["auto_lbl_conf"] 

auto_lbl_conf['threshold_estimation'] = 'val_estimate'
#auto_lbl_conf['threshold_estimation'] = 'fixed'
#auto_lbl_conf['fixed_threshold'] = 0.6


pseudo_lbl_conf = conf["pseudo_lbl_conf"] 

#pseudo_lbl_conf['threshold_estimation'] = 'val_estimate'
pseudo_lbl_conf['threshold_estimation'] = 'val_estimate'

#pseudo_lbl_conf['fixed_threshold'] = 0.95

In [4]:
set_seed(conf['random_seed'])

dm = DataManager(conf,logger,lib=conf['model_conf']['lib'])

logger.info('Loaded dataset {}'.format(conf['data_conf']['name']))
logger.info(f' std_train_size : {len(dm.ds_std_train)} and  std_val_size: {len(dm.ds_std_val)}')

st = SelfTraining(conf,dm,logger)

st.init()

lst_epoch_out = st.run_al_loop()

logger.info('Self Training Loop Done')

Files already downloaded and verified
Files already downloaded and verified
[04/02/2024 06:19:46 PM : DEBUG : data_manag : ] : torch.Size([40000, 3, 32, 32])
[04/02/2024 06:19:46 PM : INFO  : data_manag : ] : Loaded dataset cifar10
[04/02/2024 06:19:46 PM : INFO  : data_manag : ] : Std train size: 40000 and Std. Val. Size:8000
[04/02/2024 06:19:46 PM : INFO  : 187494373 : ] : Loaded dataset cifar10
[04/02/2024 06:19:46 PM : INFO  : 187494373 : ] :  std_train_size : 40000 and  std_val_size: 8000
[04/02/2024 06:19:46 PM : INFO  : self_train : ] : xxxxxxxxxxxxxxxxxxxxx  Running TBAL with evaluation (auto-labeling) on the full unlabeled data   xxxxxxxxxxxxxxxxxxxxx
[04/02/2024 06:19:46 PM : DEBUG : self_train : ] : Unlabeled count in check_stop_criterion 40000
[04/02/2024 06:19:46 PM : DEBUG : self_train : ] : cur_query_count= 0 and max_query_count=10000
[04/02/2024 06:19:46 PM : INFO  : self_train : ] : ===========================================================================
[04/02/202

In [5]:

auto_labeler = AutoLabeling(conf,dm,st.cur_clf,logger)
out = auto_labeler.run()
dm.get_auto_labeling_counts()

[04/02/2024 06:51:35 PM : INFO  : auto_label : ] : xxxxxxxxxxxxxxxxxxxxx  Auto-labeling actual remaining unlabeled data  xxxxxxxxxxxxxxxxxxxxx
[04/02/2024 06:51:35 PM : INFO  : auto_label : ] : ========================= Begin Auto-Labeling selective ==========================
[04/02/2024 06:51:35 PM : DEBUG : auto_label : ] : Auto Labeling Conf : {'method_name': 'selective', 'score_type': 'confidence', 'class_wise': 'independent', 'auto_label_err_threshold': 0.05, 'C_1': 0.25, 'ucb': 'sigma', 'threshold_estimation': 'val_estimate', 'fast': True}
[04/02/2024 06:51:35 PM : INFO  : auto_label : ] : Number of unlabeled points : 30000
[04/02/2024 06:51:35 PM : INFO  : data_manag : ] :  Cur calib ds size : 0
[04/02/2024 06:51:35 PM : INFO  : auto_label : ] : Using number of validation points : 8000
[04/02/2024 06:51:35 PM : DEBUG : auto_label : ] : Expected Calibration Error on Validation set : 0.08659307582117617
[04/02/2024 06:51:35 PM : INFO  : auto_label : ] : Determining Thresholds : Cl

{'auto_labeled_acc': 0.9809179989685405,
 'coverage_1': 0.09695,
 'coverage_2': 0}